**<font size='6' color='aqua'>ch3. 연관분석</font>**
- pip install apyori

In [1]:
! pip install apyori

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5975 sha256=1893cc0ae523d81482b6981adc4a4a7096261891fa2ae267bced07fbbd216d4a
  Stored in directory: c:\users\4545\appdata\local\pip\cache\wheels\84\45\a4\8ade6576f75410d8162c6da1de0aa9df56c16c711acab5a813
Successfully built apyori


# 1. 연관분석 개요
- 데이터들 사이에 자주 발생하는 속성을 찾고, 그 속성들 사이에 연관성이 어느 정도 있는지 분석하는 방법
- 활용분야 : 상품진열, 사기보험적발, 신상품 카테고리 구성....

```
{조건} -> {결과}
연관분석과 관련 지표
1. 지지도(support) : 전체 데이터 중에 조건결과 항목들의 포함된 거래 비율
    (X, Y)의 항목수 / 전체 데이터수 ex.0.2
2. 신뢰도(confidence) : 조건이 발생했을 때, 결과가 동시에 일어날 확률
    (X, Y)의 항목수 / 조건항목수 ex.0.5
3. 향상도(lift) : 우현히 발생한 규칙은 아닌지 확인
    조건결과 지지도 / (조건지지도)*(결과지지도)
    <1 : 음의 상관 관계
    >1 : 양의 상관관계
                           지지도        신뢰도          향상도
{오렌지주스} -> {와인}     1/5(0.2)      1/2(0.5)      0.2/0.4*0.6 = 0.2/0.24 = 0.833
```

In [3]:
%ls data\cf_basket.csv

 D 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 149E-6D0A

 D:\ai\source\10_자연어처리\data 디렉터리

2024-12-30  오전 09:03               119 cf_basket.csv
               1개 파일                 119 바이트
               0개 디렉터리  472,588,083,200 바이트 남음


# 2. 연관분석 구현

In [5]:
# 트랜젝션 데이터 가져오기
import csv
transaction = []
with open('data/cf_basket.csv', 'r', encoding='utf-8') as f:
    csvdata = csv.reader(f)
    for row in csvdata:
        transaction.append(row)
transaction

[['소주', '콜라', '와인'],
 ['소주', '오렌지주스', '콜라'],
 ['맥주', '콜라', '와인'],
 ['소주', '콜라', '맥주'],
 ['오렌지주스', '와인']]

In [7]:
from apyori import apriori
rules = apriori(transaction, min_support=0.15, min_confidence=0.1)
rules = list(rules)
len(rules)

18

In [8]:
rules[15]
# items=frozenset({'와인', '콜라', '맥주'}), support=0.2 : 이 세트가 함께 나타날 확률
# items_base : 조건(lhs)
# items_add : 결과(rhs)

RelationRecord(items=frozenset({'콜라', '맥주', '와인'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'콜라', '맥주', '와인'}), confidence=0.2, lift=1.0), OrderedStatistic(items_base=frozenset({'맥주'}), items_add=frozenset({'콜라', '와인'}), confidence=0.5, lift=1.25), OrderedStatistic(items_base=frozenset({'와인'}), items_add=frozenset({'콜라', '맥주'}), confidence=0.33333333333333337, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라'}), items_add=frozenset({'맥주', '와인'}), confidence=0.25, lift=1.25), OrderedStatistic(items_base=frozenset({'맥주', '와인'}), items_add=frozenset({'콜라'}), confidence=1.0, lift=1.25), OrderedStatistic(items_base=frozenset({'콜라', '맥주'}), items_add=frozenset({'와인'}), confidence=0.5, lift=0.8333333333333334), OrderedStatistic(items_base=frozenset({'콜라', '와인'}), items_add=frozenset({'맥주'}), confidence=0.5, lift=1.25)])

In [13]:
row = rules[15]
support = row[1]
ordered_st = row[2]
for item in ordered_st:
    lhs = ', '.join([x for x in item[0]])
    rhs = ', '.join([x for x in item[1]])
    confidence = item[2]
    lift = item[3]
    if lift>1 :
        print("{} => {} \t {} \t {} \t {}".format(lhs, rhs, support, confidence, lift))

맥주 => 콜라, 와인 	 0.2 	 0.5 	 1.25
콜라 => 맥주, 와인 	 0.2 	 0.25 	 1.25
맥주, 와인 => 콜라 	 0.2 	 1.0 	 1.25
콜라, 와인 => 맥주 	 0.2 	 0.5 	 1.25


In [37]:
print('조건 -> 결과 \t\t\t 지지도\t신뢰도\t향상도')
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift>1 :
            if (len(lhs)<=2) & (len(rhs)<=2):
                print("{} => {} \t\t\t {} \t {:.2f} \t {:.2f}".format(lhs, rhs, support, confidence, lift))
            else:
                print("{} => {} \t\t {} \t {:.2f} \t {:.2f}".format(lhs, rhs, support, confidence, lift))

조건 -> 결과 			 지지도	신뢰도	향상도
맥주 => 콜라 			 0.4 	 1.00 	 1.25
콜라 => 맥주 			 0.4 	 0.50 	 1.25
소주 => 콜라 			 0.6 	 1.00 	 1.25
콜라 => 소주 			 0.6 	 0.75 	 1.25
콜라 => 맥주, 소주 		 0.2 	 0.25 	 1.25
맥주, 소주 => 콜라 		 0.2 	 1.00 	 1.25
맥주 => 콜라, 와인 		 0.2 	 0.50 	 1.25
콜라 => 맥주, 와인 		 0.2 	 0.25 	 1.25
맥주, 와인 => 콜라 		 0.2 	 1.00 	 1.25
콜라, 와인 => 맥주 		 0.2 	 0.50 	 1.25
소주 => 콜라, 오렌지주스 		 0.2 	 0.33 	 1.67
콜라 => 오렌지주스, 소주 		 0.2 	 0.25 	 1.25
오렌지주스, 소주 => 콜라 		 0.2 	 1.00 	 1.25
콜라, 오렌지주스 => 소주 		 0.2 	 1.00 	 1.67
콜라 => 와인, 소주 		 0.2 	 0.25 	 1.25
와인, 소주 => 콜라 		 0.2 	 1.00 	 1.25


In [38]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','support','confidence','lift'])
index=0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift>1 :
            rules_df.loc[index] = [lhs, rhs, support, confidence, lift]            
            index += 1
rules_df.sort_values(by=['confidence', 'lift'], ascending=False)

,lhs,rhs,support,confidence,lift
13,"콜라, 오렌지주스",소주,0.2,1.000000,1.666667
0,맥주,콜라,0.4,1.000000,1.250000
2,소주,콜라,0.6,1.000000,1.250000
5,"맥주, 소주",콜라,0.2,1.000000,1.250000
8,"맥주, 와인",콜라,0.2,1.000000,1.250000
12,"오렌지주스, 소주",콜라,0.2,1.000000,1.250000
15,"와인, 소주",콜라,0.2,1.000000,1.250000
3,콜라,소주,0.6,0.750000,1.250000
1,콜라,맥주,0.4,0.500000,1.250000
6,맥주,"콜라, 와인",0.2,0.500000,1.250000


# 3. 뉴스 연관 분석
- http://fs.jtbc.joins.com/RSS/economy.xml 기사 검색후 -> 명사 추출(기자, 앵커 제외) -> 연관분석(min_support=0.15, min_confident=0.1)

In [5]:
import requests
from bs4 import BeautifulSoup
from mecab import MeCab
from gensim.models import Word2Vec
from apyori import apriori
rss_url = "http://fs.jtbc.joins.com/RSS/economy.xml"
jtbc_economy = requests.get(rss_url)
soup = BeautifulSoup(jtbc_economy.content, 'xml')
link_ele = soup.select('item link')
link_list = [link.text for link in link_ele]
mecab = MeCab()
news = []
for link in link_list:
    response = requests.get(link)
    news_soup = BeautifulSoup(response.content, 'html.parser')
    title = news_soup.select_one('title').text
    content = str(news_soup.select_one('meta[name="description"]'))
    content = content.replace('<meta content="','').replace('" name ="description"/>','')
    noun_list = [word for word, tag in mecab.pos(title + ' ' + content) if tag in ('NNG', 'NNP')]
    noun_list = [noun for noun in noun_list if noun not in ('기자', '앵커')]
    news.append(noun_list)

rules = apriori(news, min_support=0.15, min_confidence=0.1)
rules = list(rules)
print(news)

[['둔촌', '주공', '전세', '홍수', '예상', '눈치', '게임', '시작', '뉴스', '단군', '최대', '규모', '재건축', '둔촌', '주공', '아파트', '입주', '다음', '달', '시작', '규모', '입주', '전세', '매물', '현장', '분위기', '이상', '다음', '달', '세대', '입주', '시작', '서울', '둔촌동', '올림픽', '파크', '포레', '온', '국내', '건축', '가운데', '역대', '최대', '규모', '이용택', '공인', '중개사', '공개', '행사', '이후', '가량', '판단', '연령', '세대', '관심', '단지', '입주', '시기', '전', '전세', '물량', '지도', '관심', '전', '세대', '입주', '서울', '헬리오', '시티', '당시', '전세', '매매', '가격', '동반', '하락', '이번', '현장', '김복수', '공인', '중개사', '일반', '아파트', '입주', '정도', '전세', '보통', '시장', '모습', '전세', '입주', '전세', '본인', '가격', '거주', '의무', '폐지', '신축', '선호', '현상', '가세', '전세', '매물', '자체', '부족', '인터넷', '매물', '세대', '최근', '정부', '대출', '관리', '전세', '대출', '규제', '점', '한몫', '권대', '서강대', '부동산', '학과', '교수', '전세', '대출', '세입자', '물량', '전세', '대출', '실입', '주', '현상', '임대', '물량', '상대', '전세', '집', '주인', '주변', '전셋값', '신축', '중심', '추세', '함영진', '직방', '빅', '데이터', '랩', '장', '규모', '입', '주장', '전세', '가격', '하락', '매물', '유통', '기대', '영상', '취재', '승규', '영상', '편집', '김지우'], ['실제', '캠', '핑장',

In [6]:
row = rules[15]
support = row[1]
ordered_st = row[2]
for item in ordered_st:
    lhs = ', '.join([x for x in item[0]])
    rhs = ', '.join([x for x in item[1]])
    confidence = item[2]
    lift = item[3]
    if lift>1 :
        print("{} => {} \t {} \t {} \t {}".format(lhs, rhs, support, confidence, lift))

In [8]:
import pandas as pd
rules_df = pd.DataFrame(None, columns=['lhs','rhs','support','confidence','lift'])
index=0
for row in rules:
    support = row[1]
    ordered_st = row[2]
    for item in ordered_st:
        lhs = ', '.join([x for x in item[0]])
        rhs = ', '.join([x for x in item[1]])
        confidence = item[2]
        lift = item[3]
        if lift>1 :
            rules_df.loc[index] = [lhs, rhs, support, confidence, lift]            
            index += 1
rules_df.sort_values(by=['confidence', 'lift'], ascending=False)

,lhs,rhs,support,confidence,lift
6,사실,상태,0.15,1.00,6.666667
7,상태,사실,0.15,1.00,6.666667
8,사실,업체,0.15,1.00,6.666667
9,업체,사실,0.15,1.00,6.666667
10,상태,업체,0.15,1.00,6.666667
11,업체,상태,0.15,1.00,6.666667
24,사실,"뉴스, 상태",0.15,1.00,6.666667
25,상태,"뉴스, 사실",0.15,1.00,6.666667
26,"뉴스, 사실",상태,0.15,1.00,6.666667
27,"뉴스, 상태",사실,0.15,1.00,6.666667
